In [18]:
import requests
from urllib.parse import urlparse, parse_qs
import pandas as pd
import math

In [54]:
def sum(df,row,n):
    return df['intensityScoreNormalized'][row.name:row.name+n].sum() 
    # if row.name != len(df['intensityScoreNormalized'])-n : 
    #     return df['intensityScoreNormalized'][row.name:row.name+n].sum() 
    # else :
    #     return 0


In [67]:
def get_most_viewed(yt_link):

    # Parse the URL
    parsed_url = urlparse(yt_link)

    # Extract the video ID from the query parameters
    video_id = parse_qs(parsed_url.query).get("v", [])[0] if "v" in parse_qs(parsed_url.query) else None

    if not video_id:
        print("Unable to extract video ID from the URL.")
   
    api_url = "https://yt.lemnoslife.com/videos"
    params = {
        "part": "mostReplayed",
        "id": video_id
    }

    response = requests.get(api_url, params=params)

    if response.status_code == 200:
        # The request was successful
        data = response.json()
        markers = data["items"][0]["mostReplayed"]["markers"]
        df = pd.DataFrame(markers)
        df = df.iloc[1:-1]
        #most_viewed = [element for element in markers if element["intensityScoreNormalized"] > 0.8]
        pas = df["startMillis"][1]
        rows_3min = math.ceil(180000/pas)
        df['sum'] = df.apply(lambda row: df['intensityScoreNormalized'][row.name:row.name+rows_3min+1].sum(), axis=1)
        max_score_index = df['sum'].idxmax()
        best_start = df.loc[max_score_index, 'startMillis']
        return best_start
    else:
        # The request failed
        print(f"Error: {response.status_code} - {response.text}")


In [68]:

get_most_viewed("https://www.youtube.com/watch?v=dWbPdqns5Xk")

,startMillis,intensityScoreNormalized,sum
1,27200,0.320402,2.670595
2,54400,0.216156,2.637212
3,81600,0.124508,3.008653
4,108800,0.293796,2.890877
5,136000,0.373424,2.842811
6,163200,0.855351,2.191008
7,190400,0.319146,2.107251
8,217600,0.272970,2.133775
9,244800,0.215244,2.101017
10,272000,0.182774,2.158987


516800
